# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-04-03, 15:41:32

   -> Summany DataFrame : 132 rows, 20 columns

TDMDBPR : Disconnected


In [3]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250401,DB1R000100,Prepay Revenue,2025-04-01,20240528,20250401,"6,061,961,844","11,004,711,396","10,996,613,790","10,996,613,790",0,"102,941,844",4,172,18827
1,20250401,DB2R000100,Postpaid Revenue,2025-04-01,20240528,20250401,"9,470,202,377","17,136,299,472","17,122,044,743","17,122,044,743",0,"271,189,970",4,172,19037
2,20250401,DB2R010100,Postpaid Revenue B2C,2025-04-01,20240528,20250401,"8,684,970,103","15,744,997,149","15,732,538,124","15,732,538,124",0,"255,149,750",4,172,19032
3,20250401,DB2R020100,Postpaid Revenue B2B,2025-04-01,20240528,20250401,"785,232,273","1,391,302,322","1,389,506,619","1,389,506,619",0,"16,040,220",4,172,18998


In [4]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250401,DB1S000500,Prepay Usage Subs,2025-04-01,20240528,20250401,"1,314,669,828","2,390,670,061","2,388,972,292","2,388,972,292",0,"23,542,689",4,172,20122
1,20250401,DB1S000600,Prepay Revenue Subs,2025-04-01,20240528,20250401,"696,433,636","1,270,899,704","1,269,896,834","1,269,896,834",0,"12,582,267",4,172,20122
2,20250401,DB2S000500,Postpaid Active Subs,2025-04-01,20240528,20250401,"646,559,396","1,178,123,797","1,177,108,749","1,177,108,749",0,"17,817,040",4,172,20122
3,20250401,DB2S010500,Postpaid Active Subs B2C,2025-04-01,20240528,20250401,"570,853,583","1,039,662,569","1,038,864,920","1,038,864,920",0,"16,066,127",4,172,20122
4,20250401,DB2S020400,Postpaid Active Sub B2B,2025-04-01,20240528,20250401,"75,705,813","138,461,228","138,243,829","138,243,829",0,"1,750,913",4,172,19921


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [5]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-04-03, 15:41:43

   -> Daily DataFrame : 968 rows, 16 columns

TDMDBPR : Disconnected


In [6]:
''' DB1R000100 : Prepay Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250401,DB1R000100,Prepay Revenue,20250401,2025-04-01,"69,765,656","139,531,311","139,399,805","139,399,805",0,"1,575,177",0,0,4,170,185


In [7]:
''' DB2R000100 : Postpaid Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250401,DB2R000100,Postpaid Revenue,20250401,2025-04-01,"7,680,029","15,360,058","15,350,005","15,350,005",0,"212,068",0,0,4,170,183


In [8]:
''' DB2R010100 : Postpaid Revenue B2C '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250401,DB2R010100,Postpaid Revenue B2C,20250401,2025-04-01,"4,707,443","9,414,885","9,407,076","9,407,076",0,"191,302",0,0,4,170,183


In [9]:
''' DB2R020100 : Postpaid Revenue B2B '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250401,DB2R020100,Postpaid Revenue B2B,20250401,2025-04-01,"2,972,587","5,945,173","5,942,928","5,942,928",0,"20,766",0,0,4,170,183


In [10]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]